# Based
- https://www.kaggle.com/code/honganzhu/cmi-piu-competition?scriptVersionId=201912528 Version44 LB0.492

 If you find this notebook useful, please upvote this and the based one.

In [1]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold, ShuffleSplit
from scipy.optimize import minimize
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import polars as pl
import polars.selectors as cs
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
import seaborn as sns

from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

from colorama import Fore, Style
from IPython.display import clear_output
import warnings
import lightgbm as lgb
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from pytorch_tabnet.tab_model import TabNetRegressor
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)

pl.Config.set_tbl_rows(-1)
pl.Config.set_tbl_cols(-1)
pl.Config.set_fmt_str_lengths(10000)

polars.config.Config

In [2]:
#from IPython.display import display as ipydisplay, HTML
#ipydisplay(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 80em; }</style>"))

In [3]:
SEED = 42
n_splits = 5

# Feature Engineering

- **Feature Selection**: The dataset contains features related to physical characteristics (e.g., BMI, Height, Weight), behavioral aspects (e.g., internet usage), and fitness data (e.g., endurance time). 
- **Categorical Feature Encoding**: Categorical features are mapped to numerical values using custom mappings for each unique category within the dataset. This ensures compatibility with machine learning algorithms that require numerical input.
- **Time Series Aggregation**: Time series statistics (e.g., mean, standard deviation) from the actigraphy data are computed and merged into the main dataset to create additional features for model training.


In [4]:
path = "I:/Kaggle/child-mind-institute-problematic-internet-use/"

In [5]:
train = pd.read_csv(path + 'train.csv', dtype={'id': str})
test = pd.read_csv(path + 'test.csv', dtype={'id': str})
sample = pd.read_csv(path + 'sample_submission.csv', dtype={'id': str})

In [6]:
print(train.shape)
train.head()

(3960, 82)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [7]:
print(test.shape)
test.head()

(20, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [8]:
sample

id  sii
0   00008ff9    0
1   000fd460    1
2   00105258    2
3   00115b9f    3
4   0016bb22    0
5   001f3379    1
6   0038ba98    2
7   0068a485    3
8   0069fbed    0
9   0083e397    1
10  0087dd65    2
11  00abe655    3
12  00ae59c9    0
13  00af6387    1
14  00bd4359    2
15  00c0cd71    3
16  00d56d4b    0
17  00d9913d    1
18  00e6167c    2
19  00ebc35d    3

In [9]:
data_dict = pd.read_csv(path + 'data_dictionary.csv')
print(data_dict.shape)
data_dict

(81, 6)


Instrument  \
0                                      Identifier   
1                                    Demographics   
2                                    Demographics   
3                                    Demographics   
4              Children's Global Assessment Scale   
5              Children's Global Assessment Scale   
6                               Physical Measures   
7                               Physical Measures   
8                               Physical Measures   
9                               Physical Measures   
10                              Physical Measures   
11                              Physical Measures   
12                              Physical Measures   
13                              Physical Measures   
14               FitnessGram Vitals and Treadmill   
15               FitnessGram Vitals and Treadmill   
16               FitnessGram Vitals and Treadmill   
17               FitnessGram Vitals and Treadmill   
18                              FitnessGram Child   
19                              FitnessGram Child   
20                              FitnessGram Child   
21                              FitnessGram Child   
22                              FitnessGram Child   
23                              FitnessGram Child   
24                              FitnessGram Child   
25                              FitnessGram Child   
26                              FitnessGram Child   
27                              FitnessGram Child   
28                              FitnessGram Child   
29                              FitnessGram Child   
30                              FitnessGram Child   
31                              FitnessGram Child   
32                              FitnessGram Child   
33                Bio-electric Impedance Analysis   
34                Bio-electric Impedance Analysis   
35                Bio-electric Impedance Analysis   
36                Bio-electric Impedance Analysis   
37                Bio-electric Impedance Analysis   
38                Bio-electric Impedance Analysis   
39                Bio-electric Impedance Analysis   
40                Bio-electric Impedance Analysis   
41                Bio-electric Impedance Analysis   
42                Bio-electric Impedance Analysis   
43                Bio-electric Impedance Analysis   
44                Bio-electric Impedance Analysis   
45                Bio-electric Impedance Analysis   
46                Bio-electric Impedance Analysis   
47                Bio-electric Impedance Analysis   
48                Bio-electric Impedance Analysis   
49                Bio-electric Impedance Analysis   
50  Physical Activity Questionnaire (Adolescents)   
51  Physical Activity Questionnaire (Adolescents)   
52     Physical Activity Questionnaire (Children)   
53     Physical Activity Questionnaire (Children)   
54           Parent-Child Internet Addiction Test   
55           Parent-Child Internet Addiction Test   
56           Parent-Child Internet Addiction Test   
57           Parent-Child Internet Addiction Test   
58           Parent-Child Internet Addiction Test   
59           Parent-Child Internet Addiction Test   
60           Parent-Child Internet Addiction Test   
61           Parent-Child Internet Addiction Test   
62           Parent-Child Internet Addiction Test   
63           Parent-Child Internet Addiction Test   
64           Parent-Child Internet Addiction Test   
65           Parent-Child Internet Addiction Test   
66           Parent-Child Internet Addiction Test   
67           Parent-Child Internet Addiction Test   
68           Parent-Child Internet Addiction Test   
69           Parent-Child Internet Addiction Test   
70           Parent-Child Internet Addiction Test   
71           Parent-Child Internet Addiction Test   
72           Parent-Child Internet Addiction Test   
73           Parent-Child Internet Addiction Test   
74           Parent-Child Internet Addiction Test   
75         

In [10]:
train['id'].nunique()

3960

In [11]:
test['id'].nunique()

20

In [12]:
train_target = train[test.columns]

In [13]:
print(train_target.shape)
train_target.head()

(3960, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [14]:
concat_df = pd.concat([train_target, test], ignore_index=True)
print(concat_df.shape)
concat_df.head()

(3980, 59)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [15]:
concat_df['total_num_nan'] = concat_df.isna().sum(axis=1)

In [16]:
concat_df['total_num_nan'].describe()

count    3980.000000
mean       26.142462
std        17.257902
min         3.000000
25%         9.000000
50%        24.000000
75%        41.000000
max        55.000000
Name: total_num_nan, dtype: float64

In [17]:
nan_df = concat_df[concat_df['total_num_nan'] == concat_df['total_num_nan'].max()]
print(nan_df.shape)
nan_df.head()

(199, 60)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
42   023e1b6a                    Summer               13                0   
67   040d103f                    Spring                8                1   
84   04ccb838                    Summer                7                0   
107  05dc60ef                    Spring               12                1   
148  0882b065                    Summer               11                0   

    CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
42          NaN              NaN             NaN           NaN   
67          NaN              NaN             NaN           NaN   
84          NaN              NaN             NaN           NaN   
107         NaN              NaN             NaN           NaN   
148         NaN              NaN             NaN           NaN   

     Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
42               NaN              NaN                           NaN   
67               NaN              NaN                           NaN   
84               NaN              NaN                           NaN   
107              NaN              NaN                           NaN   
148              NaN              NaN                           NaN   

     Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
42                     NaN                 NaN                   NaN   
67                     NaN                 NaN                   NaN   
84                     NaN                 NaN                   NaN   
107                    NaN                 NaN                   NaN   
148                    NaN                 NaN                   NaN   

    Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
42                       NaN                          NaN   
67                       NaN                          NaN   
84                       NaN                          NaN   
107                      NaN                          NaN   
148                      NaN                          NaN   

     Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
42                           NaN                         NaN        NaN   
67                           NaN                         NaN        NaN   
84                           NaN                         NaN        NaN   
107                          NaN                         NaN        NaN   
148                          NaN                         NaN        NaN   

     FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
42          NaN              NaN           NaN                NaN   
67          NaN              NaN           NaN                NaN   
84          NaN              NaN           NaN                NaN   
107         NaN              NaN           NaN                NaN   
148         NaN              NaN           NaN                NaN   

     FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
42           NaN               NaN         NaN              NaN          NaN   
67           NaN               NaN         NaN              NaN          NaN   
84           NaN               NaN         NaN              NaN          NaN   
107          NaN               NaN         NaN              NaN          NaN   
148          NaN               NaN         NaN              NaN          NaN   

     FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
42                NaN          NaN               NaN         NaN   
67                NaN          NaN               NaN         NaN   
84                NaN          NaN               NaN         NaN   
107               NaN          NaN               NaN         NaN   
148               NaN          NaN               NaN         NaN   

     FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
42               NaN        NaN                         NaN          NaN   
67              

In [18]:
for col in concat_df.columns:
    if col == 'id':
        continue
    new_col_name = col + '_isnan'
    concat_df[new_col_name] = concat_df[col].isna().astype(int)

In [19]:
print(concat_df.shape)
concat_df.head()

(3980, 119)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [20]:
instruments = data_dict['Instrument'].unique().tolist()
instruments.remove('Identifier')
print(len(instruments))
instruments

11


['Demographics',
 "Children's Global Assessment Scale",
 'Physical Measures',
 'FitnessGram Vitals and Treadmill',
 'FitnessGram Child',
 'Bio-electric Impedance Analysis',
 'Physical Activity Questionnaire (Adolescents)',
 'Physical Activity Questionnaire (Children)',
 'Parent-Child Internet Addiction Test',
 'Sleep Disturbance Scale',
 'Internet Use']

In [21]:
for instrument in instruments:
    if 'Parent-Child Internet Addiction Test' == instrument:
        continue
    instrument_df = data_dict[data_dict['Instrument'] == instrument]
    field_list = instrument_df['Field'].unique().tolist()
    concat_df[instrument + '_num_nan'] = concat_df[field_list].isna().sum(axis=1)    

In [22]:
print(concat_df.shape)
concat_df.head()

(3980, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [23]:
train_parquet_id_folders_list = os.listdir(path + 'series_train.parquet/')
print(len(train_parquet_id_folders_list))
train_parquet_id_folders_list

996


['id=00115b9f',
 'id=001f3379',
 'id=00f332d1',
 'id=01085eb3',
 'id=012cadd8',
 'id=012e3869',
 'id=029a19c9',
 'id=02cebf33',
 'id=02cf7384',
 'id=035c96dd',
 'id=03a9019b',
 'id=0417c91e',
 'id=045a0a94',
 'id=04afb6f9',
 'id=04bb1a76',
 'id=04cb2c30',
 'id=04d06a9c',
 'id=04f094a8',
 'id=051680a0',
 'id=055156e2',
 'id=059eed01',
 'id=05bbed1b',
 'id=05db1b9b',
 'id=05e94f88',
 'id=063b16fc',
 'id=064e8da5',
 'id=0668373f',
 'id=067b9287',
 'id=06c8b2fb',
 'id=06eb8adb',
 'id=070386b2',
 'id=07048067',
 'id=0720320e',
 'id=0745c390',
 'id=080698b7',
 'id=082df556',
 'id=08577722',
 'id=08660d12',
 'id=089a6b19',
 'id=09331bb8',
 'id=0983654b',
 'id=099e3614',
 'id=09b29113',
 'id=0a418b57',
 'id=0a431608',
 'id=0b4014f0',
 'id=0b50f3fa',
 'id=0b518637',
 'id=0b7d7aec',
 'id=0b7d9da6',
 'id=0bd89878',
 'id=0bfe4fc9',
 'id=0c4e2d68',
 'id=0c4eabac',
 'id=0c98f822',
 'id=0cbe4e54',
 'id=0d01bbf2',
 'id=0d279d77',
 'id=0d4800c8',
 'id=0d4fa8f4',
 'id=0d570d3e',
 'id=0d924604',
 'id=0e3

In [24]:
train_parquet_id_list = [v.replace('id=', '') for v in train_parquet_id_folders_list]
print(len(train_parquet_id_list))
train_parquet_id_list

996


['00115b9f',
 '001f3379',
 '00f332d1',
 '01085eb3',
 '012cadd8',
 '012e3869',
 '029a19c9',
 '02cebf33',
 '02cf7384',
 '035c96dd',
 '03a9019b',
 '0417c91e',
 '045a0a94',
 '04afb6f9',
 '04bb1a76',
 '04cb2c30',
 '04d06a9c',
 '04f094a8',
 '051680a0',
 '055156e2',
 '059eed01',
 '05bbed1b',
 '05db1b9b',
 '05e94f88',
 '063b16fc',
 '064e8da5',
 '0668373f',
 '067b9287',
 '06c8b2fb',
 '06eb8adb',
 '070386b2',
 '07048067',
 '0720320e',
 '0745c390',
 '080698b7',
 '082df556',
 '08577722',
 '08660d12',
 '089a6b19',
 '09331bb8',
 '0983654b',
 '099e3614',
 '09b29113',
 '0a418b57',
 '0a431608',
 '0b4014f0',
 '0b50f3fa',
 '0b518637',
 '0b7d7aec',
 '0b7d9da6',
 '0bd89878',
 '0bfe4fc9',
 '0c4e2d68',
 '0c4eabac',
 '0c98f822',
 '0cbe4e54',
 '0d01bbf2',
 '0d279d77',
 '0d4800c8',
 '0d4fa8f4',
 '0d570d3e',
 '0d924604',
 '0e3b0bd2',
 '0ec403df',
 '0ed2a3c8',
 '0eddd8e5',
 '0f6b71c4',
 '0fbb3ca5',
 '101774f0',
 '101c27f1',
 '108b8659',
 '10ad8c48',
 '10e46254',
 '113f5798',
 '11739de2',
 '119d7562',
 '119e181e',

In [25]:
test_parquet_id_folders_list = os.listdir(path + 'series_test.parquet/')
print(len(test_parquet_id_folders_list))
test_parquet_id_folders_list

2


['id=00115b9f', 'id=001f3379']

In [26]:
test_parquet_id_list = [v.replace('id=', '') for v in test_parquet_id_folders_list]
print(len(test_parquet_id_list))
test_parquet_id_list

2


['00115b9f', '001f3379']

In [27]:
train_temp_df = concat_df.head(train.shape[0])
print(train_temp_df.shape)
train_temp_df.head()

(3960, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [28]:
test_temp_df = concat_df.tail(test.shape[0])
print(test_temp_df.shape)
test_temp_df.head()

(20, 129)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
3960  00008ff9                      Fall                5                0   
3961  000fd460                    Summer                9                0   
3962  00105258                    Summer               10                1   
3963  00115b9f                    Winter                9                0   
3964  0016bb22                    Spring               18                1   

     CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
3960      Winter             51.0            Fall     16.877316   
3961         NaN              NaN            Fall     14.035590   
3962        Fall             71.0            Fall     16.648696   
3963        Fall             71.0          Summer     18.292347   
3964      Summer              NaN             NaN           NaN   

      Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
3960             46.0             50.8                           NaN   
3961             48.0             46.0                          22.0   
3962             56.5             75.6                           NaN   
3963             56.0             81.6                           NaN   
3964              NaN              NaN                           NaN   

      Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
3960                    NaN                 NaN                   NaN   
3961                   75.0                70.0                 122.0   
3962                   65.0                94.0                 117.0   
3963                   60.0                97.0                 117.0   
3964                    NaN                 NaN                   NaN   

     Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
3960                      NaN                          NaN   
3961                      NaN                          NaN   
3962                     Fall                          5.0   
3963                   Summer                          6.0   
3964                      NaN                          NaN   

      Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
3960                          NaN                         NaN       Fall   
3961                          NaN                         NaN       Fall   
3962                          7.0                        33.0       Fall   
3963                          9.0                        37.0     Summer   
3964                          NaN                         NaN        NaN   

      FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
3960         0.0              0.0           NaN                NaN   
3961         3.0              0.0           NaN                NaN   
3962        20.0              1.0          10.2                1.0   
3963        18.0              1.0           NaN                NaN   
3964         NaN              NaN           NaN                NaN   

      FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
3960          NaN               NaN         0.0              0.0          7.0   
3961          NaN               NaN         5.0              0.0         11.0   
3962         14.7               2.0         7.0              1.0         10.0   
3963          NaN               NaN         5.0              0.0          7.0   
3964          NaN               NaN         NaN              NaN          NaN   

      FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
3960               0.0          6.0               0.0         6.0   
3961               1.0         11.0               1.0         3.0   
3962               1.0         10.0               1.0         5.0   
3963               0.0          7.0               0.0         7.0   
3964               NaN          NaN               NaN         NaN   

      FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
3960              1.0       Fall      

In [29]:
train_temp_df['has_parquet'] = train_temp_df['id'].isin(train_parquet_id_list).astype(int)
print(train_temp_df.shape)
train_temp_df.head()

(3960, 130)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
0  00008ff9                      Fall                5                0   
1  000fd460                    Summer                9                0   
2  00105258                    Summer               10                1   
3  00115b9f                    Winter                9                0   
4  0016bb22                    Spring               18                1   

  CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  Physical-Height  \
0      Winter             51.0            Fall     16.877316             46.0   
1         NaN              NaN            Fall     14.035590             48.0   
2        Fall             71.0            Fall     16.648696             56.5   
3        Fall             71.0          Summer     18.292347             56.0   
4      Summer              NaN             NaN           NaN              NaN   

   Physical-Weight  Physical-Waist_Circumference  Physical-Diastolic_BP  \
0             50.8                           NaN                    NaN   
1             46.0                          22.0                   75.0   
2             75.6                           NaN                   65.0   
3             81.6                           NaN                   60.0   
4              NaN                           NaN                    NaN   

   Physical-HeartRate  Physical-Systolic_BP Fitness_Endurance-Season  \
0                 NaN                   NaN                      NaN   
1                70.0                 122.0                      NaN   
2                94.0                 117.0                     Fall   
3                97.0                 117.0                   Summer   
4                 NaN                   NaN                      NaN   

   Fitness_Endurance-Max_Stage  Fitness_Endurance-Time_Mins  \
0                          NaN                          NaN   
1                          NaN                          NaN   
2                          5.0                          7.0   
3                          6.0                          9.0   
4                          NaN                          NaN   

   Fitness_Endurance-Time_Sec FGC-Season  FGC-FGC_CU  FGC-FGC_CU_Zone  \
0                         NaN       Fall         0.0              0.0   
1                         NaN       Fall         3.0              0.0   
2                        33.0       Fall        20.0              1.0   
3                        37.0     Summer        18.0              1.0   
4                         NaN        NaN         NaN              NaN   

   FGC-FGC_GSND  FGC-FGC_GSND_Zone  FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  \
0           NaN                NaN          NaN               NaN         0.0   
1           NaN                NaN          NaN               NaN         5.0   
2          10.2                1.0         14.7               2.0         7.0   
3           NaN                NaN          NaN               NaN         5.0   
4           NaN                NaN          NaN               NaN         NaN   

   FGC-FGC_PU_Zone  FGC-FGC_SRL  FGC-FGC_SRL_Zone  FGC-FGC_SRR  \
0              0.0          7.0               0.0          6.0   
1              0.0         11.0               1.0         11.0   
2              1.0         10.0               1.0         10.0   
3              0.0          7.0               0.0          7.0   
4              NaN          NaN               NaN          NaN   

   FGC-FGC_SRR_Zone  FGC-FGC_TL  FGC-FGC_TL_Zone BIA-Season  \
0               0.0         6.0              1.0       Fall   
1               1.0         3.0              0.0     Winter   
2               1.0         5.0              0.0        NaN   
3               0.0         7.0              1.0     Summer   
4               NaN         NaN              NaN        NaN   

   BIA-BIA_Activity_Level_num  BIA-BIA_BMC  BIA-BIA_BMI  BIA-BIA_BMR  \
0                         2.0     

In [30]:
test_temp_df['has_parquet'] = test_temp_df['id'].isin(test_parquet_id_list).astype(int)
print(test_temp_df.shape)
test_temp_df.head()

(20, 130)


id Basic_Demos-Enroll_Season  Basic_Demos-Age  Basic_Demos-Sex  \
3960  00008ff9                      Fall                5                0   
3961  000fd460                    Summer                9                0   
3962  00105258                    Summer               10                1   
3963  00115b9f                    Winter                9                0   
3964  0016bb22                    Spring               18                1   

     CGAS-Season  CGAS-CGAS_Score Physical-Season  Physical-BMI  \
3960      Winter             51.0            Fall     16.877316   
3961         NaN              NaN            Fall     14.035590   
3962        Fall             71.0            Fall     16.648696   
3963        Fall             71.0          Summer     18.292347   
3964      Summer              NaN             NaN           NaN   

      Physical-Height  Physical-Weight  Physical-Waist_Circumference  \
3960             46.0             50.8                           NaN   
3961             48.0             46.0                          22.0   
3962             56.5             75.6                           NaN   
3963             56.0             81.6                           NaN   
3964              NaN              NaN                           NaN   

      Physical-Diastolic_BP  Physical-HeartRate  Physical-Systolic_BP  \
3960                    NaN                 NaN                   NaN   
3961                   75.0                70.0                 122.0   
3962                   65.0                94.0                 117.0   
3963                   60.0                97.0                 117.0   
3964                    NaN                 NaN                   NaN   

     Fitness_Endurance-Season  Fitness_Endurance-Max_Stage  \
3960                      NaN                          NaN   
3961                      NaN                          NaN   
3962                     Fall                          5.0   
3963                   Summer                          6.0   
3964                      NaN                          NaN   

      Fitness_Endurance-Time_Mins  Fitness_Endurance-Time_Sec FGC-Season  \
3960                          NaN                         NaN       Fall   
3961                          NaN                         NaN       Fall   
3962                          7.0                        33.0       Fall   
3963                          9.0                        37.0     Summer   
3964                          NaN                         NaN        NaN   

      FGC-FGC_CU  FGC-FGC_CU_Zone  FGC-FGC_GSND  FGC-FGC_GSND_Zone  \
3960         0.0              0.0           NaN                NaN   
3961         3.0              0.0           NaN                NaN   
3962        20.0              1.0          10.2                1.0   
3963        18.0              1.0           NaN                NaN   
3964         NaN              NaN           NaN                NaN   

      FGC-FGC_GSD  FGC-FGC_GSD_Zone  FGC-FGC_PU  FGC-FGC_PU_Zone  FGC-FGC_SRL  \
3960          NaN               NaN         0.0              0.0          7.0   
3961          NaN               NaN         5.0              0.0         11.0   
3962         14.7               2.0         7.0              1.0         10.0   
3963          NaN               NaN         5.0              0.0          7.0   
3964          NaN               NaN         NaN              NaN          NaN   

      FGC-FGC_SRL_Zone  FGC-FGC_SRR  FGC-FGC_SRR_Zone  FGC-FGC_TL  \
3960               0.0          6.0               0.0         6.0   
3961               1.0         11.0               1.0         3.0   
3962               1.0         10.0               1.0         5.0   
3963               0.0          7.0               0.0         7.0   
3964               NaN          NaN               NaN         NaN   

      FGC-FGC_TL_Zone BIA-Season  BIA-BIA_Activity_Level_num  BIA-BIA_BMC  \
3960              1.0       Fall      

In [31]:
# https://www.kaggle.com/code/antoninadolgorukova/cmi-piu-actigraphy-data-eda
entropy = lambda x: -(x / x.sum() * np.log(x / x.sum() + 1e-9)).sum()

In [32]:
def feat_engi(df, target_cols):
    stats_df = df[target_cols].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99]).T
    count = stats_df['count'][0]
    stats_df.drop('count', axis=1, inplace=True)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    reshaped_df['count'] = count

    return reshaped_df

In [33]:
def groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)
    
    column_names_list = []
    for idx in stats_df.index:
        for col in stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))

    reshaped_df = pd.DataFrame(stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    return reshaped_df

In [34]:
def uncommon_groupby_feat_engi(df, col_to_group, target_col):
    stats_df = pd.DataFrame(
        df.groupby(col_to_group)[target_col].describe()
        #df.groupby(col_to_group)[target_col].describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    )

    stats_df['sum'] = df.groupby(col_to_group)[target_col].sum()
    stats_df['entropy'] = df.groupby(col_to_group)[target_col].apply(entropy)

    
    #stats_stats_df = stats_df.describe(percentiles=[0.01, 0.03] + [(i+1)*0.05 for i in range(19)] + [0.97, 0.99])
    stats_stats_df = stats_df.describe()
    stats_stats_count = stats_stats_df['count'][0]
    stats_stats_df.drop('count', axis=0, inplace=True)
    stats_stats_df = stats_stats_df.T
    
    column_names_list = []
    for idx in stats_stats_df.index:
        for col in stats_stats_df.columns:
            column_names_list.append(target_col + '_' + col_to_group + '_' + str(idx) + '_' + str(col))
    
    reshaped_df = pd.DataFrame(stats_stats_df.values.reshape(-1)).T.reset_index(drop=True)
    reshaped_df.columns = column_names_list

    current_cols = reshaped_df.columns.tolist()
    count_col_name = target_col + '_' + col_to_group + '_count'
    reshaped_df[count_col_name] = stats_stats_count

    reshaped_df = reshaped_df[[count_col_name] + current_cols]

    return reshaped_df

In [35]:
def read_parquet(dataset='train'):
    parquet_id_folders_list = os.listdir(path + 'series_' + dataset + '.parquet/')
    ts_list = []
    ts_features_list = []
    for id_folder in tqdm(parquet_id_folders_list):
        iid = id_folder.replace('id=', '')
    
        parquet_files_path = path + 'series_' + dataset + '.parquet/' + id_folder + '/'
        parquet_files_list = os.listdir(parquet_files_path)
    
        dfs_list = []
        for parquet_filename in parquet_files_list:
            if '.parquet' in parquet_filename:
                parquet_df = pd.read_parquet(parquet_files_path + parquet_filename)
                dfs_list.append(parquet_df)
        parquet_concat_df = pd.concat(dfs_list, ignore_index=True)
        current_cols = parquet_concat_df.columns.tolist()
        
        parquet_concat_df['id'] = iid
        
        parquet_concat_df = parquet_concat_df[['id'] + current_cols]
        
        parquet_concat_df['day_since_wear'] = (parquet_concat_df['relative_date_PCIAT'] - parquet_concat_df['relative_date_PCIAT'].min()).astype(int)
        parquet_concat_df['time_of_day_sec'] = parquet_concat_df['time_of_day'] / 1e9
        parquet_concat_df.drop('time_of_day', axis=1, inplace=True)
        parquet_concat_df['time_of_day_min'] = parquet_concat_df['time_of_day_sec'] / 60
        parquet_concat_df['time_of_day_hour'] = parquet_concat_df['time_of_day_min'] / 60
        parquet_concat_df['time_of_day_day'] = parquet_concat_df['time_of_day_hour'] / 24
        parquet_concat_df['time_of_day_hour_window'] = parquet_concat_df['time_of_day_hour'].astype(int)
        parquet_concat_df['time_of_day_min_window'] = parquet_concat_df['time_of_day_min'].astype(int)
        parquet_concat_df['time_of_day_15_min_window'] = (parquet_concat_df['time_of_day_min'] / 15).astype(int)
        parquet_concat_df['timestamp_day'] = parquet_concat_df['day_since_wear'] + parquet_concat_df['time_of_day_day']
    
        if parquet_concat_df['timestamp_day'].nunique() != parquet_concat_df.shape[0]:
            print('yes')
    
        parquet_concat_df['timestamp_hour'] = parquet_concat_df['timestamp_day'] * 24
        parquet_concat_df['timestamp_min'] = parquet_concat_df['timestamp_day'] * 24 * 60
        parquet_concat_df['timestamp_sec'] = parquet_concat_df['timestamp_day'] * 24 * 60 * 60
        parquet_concat_df['timestamp_15_min'] = parquet_concat_df['timestamp_day'] * 24 * (60/15)
    
        parquet_concat_df['timestamp_hour_window'] = parquet_concat_df['timestamp_hour'].astype(int)
        parquet_concat_df['timestamp_min_window'] = parquet_concat_df['timestamp_min'].astype(int)
        parquet_concat_df['timestamp_15_min_window'] = parquet_concat_df['timestamp_15_min'].astype(int)
    
        battery_voltage_start = parquet_concat_df['battery_voltage'][0]
        parquet_concat_df['battery_use_since_wear'] = -(parquet_concat_df['battery_voltage'] - battery_voltage_start)
        
    
        worn_df = parquet_concat_df[parquet_concat_df['non-wear_flag'] == 0].reset_index(drop=True)
    
        
        feature_cols = ['X', 'Y', 'Z', 'enmo', 'anglez', 'light', 'battery_use_since_wear']
    
        features_df = feat_engi(worn_df, feature_cols)
    
        groupby_cols = ['weekday', 'time_of_day_hour_window']#, 'time_of_day_15_min_window']
    
        groupby_df_list = []
        for groupby_col in groupby_cols:
            for feature_col in feature_cols:
                groupby_features_df = groupby_feat_engi(worn_df, groupby_col, feature_col)
                groupby_df_list.append(groupby_features_df)
    
        groupby_concat_df = pd.concat(groupby_df_list, axis=1)
    
        uncommon_groupby_cols = ['day_since_wear', 'timestamp_hour_window']#, 'timestamp_15_min_window']#, 'timestamp_min_window']
    
        uncommon_groupby_df_list = []
        for uncommon_groupby_col in uncommon_groupby_cols:
            for unc_feature_col in feature_cols:
                uncommon_groupby_df = uncommon_groupby_feat_engi(worn_df, uncommon_groupby_col, unc_feature_col)
                uncommon_groupby_df_list.append(uncommon_groupby_df)
    
        features_concat_df = pd.concat([features_df, groupby_concat_df, uncommon_groupby_df], axis=1)
        features_current_cols = features_concat_df.columns.tolist()
        features_concat_df['id'] = iid
        features_concat_df = features_concat_df[['id'] + features_current_cols]
    
        for q in [1,2,3,4]:
            if q in worn_df['quarter'].tolist():
                features_concat_df['data_collected_in_quarter_' + str(q)] = 1
            else:
                features_concat_df['data_collected_in_quarter_' + str(q)] = 0

        features_concat_df['id'] = features_concat_df['id'].astype(str)

        features_concat_df.to_csv(path + 'series_' + dataset + '.parquet/id=' + iid + '/features.csv', index=False)
        
        ts_list.append(parquet_concat_df)
        ts_features_list.append(features_concat_df)
    return ts_list, ts_features_list

In [36]:
#train_ts_list, train_ts_features_list = read_parquet()

In [37]:
#test_ts_list, test_ts_features_list = read_parquet(dataset='test')

In [38]:
def read_features_files():
    temp_dfs_list = []
    for temp_id_folder in tqdm(train_parquet_id_folders_list):
        temp_files_list = os.listdir(path + 'series_train.parquet/' + temp_id_folder + '/')
        for temp_filename in temp_files_list:
            if '.csv' in temp_filename:
                temp_df = pd.read_csv(path + 'series_train.parquet/' + temp_id_folder + '/' + temp_filename, dtype={'id': str})
                temp_dfs_list.append(temp_df)

    return temp_dfs_list

In [39]:
train_ts_features_list = read_features_files()

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [04:43<00:00,  3.51it/s]


In [40]:
len(train_ts_features_list)

996

In [43]:
train_ts_features_list[0]['id'][0]

'00115b9f'

In [50]:
def write_files(ts_features_list):
    for i in tqdm(range(len(ts_features_list))):
        df = ts_features_list[i]
        temp_id_folder = 'id=' + df['id'][0]
        newpath = path + 'train_ts_features/' + temp_id_folder + '/'
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        df.to_csv(newpath + 'ts_features.csv', index=False)        

In [51]:
write_files(train_ts_features_list)

100%|████████████████████████████████████████████████████████████████████████████████| 996/996 [00:16<00:00, 59.92it/s]


In [ ]:
len(test_ts_features_list)

In [ ]:
train_ts_features_concat_df = pd.concat(train_ts_features_list, ignore_index=True)
print(train_ts_features_concat_df.shape)
train_ts_features_concat_df.head()

In [ ]:
'''test_ts_features_concat_df = pd.concat(test_ts_features_list, ignore_index=True)
print(test_ts_features_concat_df.shape)
test_ts_features_concat_df.head()'''

In [ ]:
train_ts_features_concat_df['id'].isin(train_parquet_id_list).sum()

In [ ]:
#test_ts_features_concat_df['id'].isin(test_parquet_id_list).sum()

In [ ]:
def remove_useless_cols(df):
    describe_df = df.describe()

    cols_to_drop = []
    for col in tqdm(describe_df.columns):
        if describe_df[col]['std'] == 0:
            cols_to_drop.append(col)

    print(len(cols_to_drop))
    
    df.drop(cols_to_drop, axis=1, inplace=True)

    return df

In [ ]:
train_ts_features_concat_df = remove_useless_cols(train_ts_features_concat_df)
print(train_ts_features_concat_df.shape)
train_ts_features_concat_df.head()

In [ ]:
'''test_ts_features_concat_df = remove_useless_cols(test_ts_features_concat_df)
print(test_ts_features_concat_df.shape)
test_ts_features_concat_df.head()'''

In [ ]:
train_temp_df['sii'] = train['sii']

In [ ]:
'''train_has_parquet_df = train_temp_df[train_temp_df['has_parquet'] == 1].reset_index(drop=True).drop('has_parquet', axis=1)
print(train_has_parquet_df.shape)
train_has_parquet_df.head()'''

In [ ]:
train_nonan_df = train_temp_df[~train_temp_df['sii'].isna()].reset_index(drop=True)
print(train_nonan_df.shape)
train_nonan_df.head()

In [ ]:
train_nonan_df.shape[1] + train_ts_features_concat_df.shape[1]

In [ ]:
train_nonan_df.isna().sum().sum()

In [ ]:
train_ts_features_concat_df.isna().sum().sum()

In [ ]:
train_nonan_df.isna().sum().sum() + train_ts_features_concat_df.isna().sum().sum()

In [ ]:
train_merge_df = train_nonan_df.merge(train_ts_features_concat_df, how='left', on='id')
print(train_merge_df.shape)
train_merge_df.head()

In [ ]:
train_merge_df.isna().sum().sum()

In [ ]:
train_merge_sii = train_merge_df['sii']
train_merge_sii

In [ ]:
train_merge_df.drop('sii', axis=1, inplace=True)
print(train_merge_df.shape)
train_merge_df.head()

In [ ]:
def remove_pciat_cols(df):
    cols_to_drop = []
    for col in df.columns:
        if 'PCIAT' in col:
            cols_to_drop.append(col)

    df.drop(cols_to_drop, axis=1, inplace=True)
    
    return df

In [ ]:
train_merge_df = remove_pciat_cols(train_merge_df)
print(train_merge_df.shape)
train_merge_df.head()

In [ ]:
for col in train_merge_df.columns:
    if 'PCIAT' in col:
        print(col)

In [ ]:
cat_cols = []
for i in range(data_dict.shape[0]):
    col = data_dict['Field'][i]
    if col == 'id':
        continue
    if (data_dict['Type'][i] == 'str') | ('categori' in data_dict['Type'][i]):
        cat_cols.append(col)

In [ ]:
ordinal_variables = ['FGC-FGC_CU_Zone', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU_Zone',
                     'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL_Zone', 'BIA-BIA_Activity_Level_num', 'BIA-BIA_Frame_num',
                     'PreInt_EduHx-computerinternet_hoursday']

ordinal_variables += [col for col in cat_cols if (('PCIAT' in col)&('Season' not in col))]
print(len(ordinal_variables))
ordinal_variables

In [ ]:
cat_cols = list(set(cat_cols) - set(ordinal_variables))
cat_cols = [col for col in cat_cols if 'PCIAT' not in col]
print(len(cat_cols))
cat_cols

In [ ]:
for col in cat_cols:
    if col in train_merge_df.columns.tolist():
        print(col)

In [ ]:
train_one_hot_df = train_merge_df.copy()

In [ ]:
def one_hot_cat_cols(df):
    for col in tqdm(cat_cols):
        df[col] = df[col].fillna('Missing').apply(str)
        unique_list = sorted(df[col].unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            df[new_col_name] = (df[col] == v).astype(int)
    
    df = df.drop(cat_cols, axis=1)

    return df

In [ ]:
train_one_hot_df = one_hot_cat_cols(train_one_hot_df)
print(train_one_hot_df.shape)
train_one_hot_df.head()

In [ ]:
# 테스트랑 같이할것

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
def feature_engineering(df):
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['Internet_Hours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    df['BMI_Internet_Hours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / df['BIA-BIA_BMI']
    df['FFMI_BFP'] = df['BIA-BIA_FFMI'] / df['BIA-BIA_Fat']
    df['FMI_BFP'] = df['BIA-BIA_FMI'] / df['BIA-BIA_Fat']
    df['LST_TBW'] = df['BIA-BIA_LST'] / df['BIA-BIA_TBW']
    df['BFP_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['BFP_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / df['Physical-Weight']
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['SMM_Height'] = df['BIA-BIA_SMM'] / df['Physical-Height']
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / df['BIA-BIA_FMI']
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / df['Physical-Weight']
    df['ICW_TBW'] = df['BIA-BIA_ICW'] / df['BIA-BIA_TBW']
    
    return df

In [ ]:
train_one_hot_df = feature_engineering(train_one_hot_df)
print(train_one_hot_df.shape)
train_one_hot_df.head()

In [ ]:
no_id_cols = [col for col in train_one_hot_df.columns if col != 'id']
print(len(no_id_cols))
no_id_cols[:5]

In [ ]:
if np.any(np.isinf(train_one_hot_df[no_id_cols])):
    print('inf in data')
    train_one_hot_df[no_id_cols] = train_one_hot_df[no_id_cols].replace([np.inf, -np.inf], np.nan)

In [ ]:
imputer = KNNImputer(n_neighbors=5)
train_one_hot_df[no_id_cols] = imputer.fit_transform(train_one_hot_df[no_id_cols])

In [ ]:
train_one_hot_df.isna().sum().sum()

In [ ]:
train_one_hot_df = remove_useless_cols(train_one_hot_df)
print(train_one_hot_df.shape)
train_one_hot_df.head()

In [ ]:
train_one_hot_df['sii'] = train_merge_sii

In [ ]:
train_final_df = train_one_hot_df.copy()

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [ ]:
# https://www.kaggle.com/code/ichigoe/lb0-494-with-tabnet
# Edited by Max (Jongyun Han)
def TrainML(model_class, train_data):#, test_data):
    X = train_data.drop(['id', 'sii'], axis=1)
    y = train_data['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    #test_preds = np.zeros((len(test_data), n_splits))

    models_list = []

    #for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
    for fold, (train_idx, test_idx) in enumerate(SKF.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train, eval_set=(X_val, y_val), plot=True)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_train, y_train_pred), 
                                  method='Nelder-Mead')
        assert train_KappaOPtimizer.success, "Train Optimization did not converge."

        train_pred_tuned = threshold_Rounder(y_train_pred, train_KappaOPtimizer.x)
        train_Kappa = quadratic_weighted_kappa(y_train, train_pred_tuned)

        print(f"----> || Train Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_Kappa:.3f}{Style.RESET_ALL}")

        val_KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y_val, y_val_pred), 
                                  method='Nelder-Mead')
        assert val_KappaOPtimizer.success, "Val Optimization did not converge."

        val_pred_tuned = threshold_Rounder(y_val_pred, val_KappaOPtimizer.x)
        val_Kappa = quadratic_weighted_kappa(y_val, val_pred_tuned)

        print(f"----> || Val Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_Kappa:.3f}{Style.RESET_ALL}")

        train_optimized_val_pred_tuned = threshold_Rounder(y_val_pred, train_KappaOPtimizer.x)
        train_optimized_val_Kappa = quadratic_weighted_kappa(y_val, train_optimized_val_pred_tuned)

        print(f"----> || Train Optimized Val QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {train_optimized_val_Kappa:.3f}{Style.RESET_ALL}")

        val_optimized_train_pred_tuned = threshold_Rounder(y_train_pred, val_KappaOPtimizer.x)
        val_optimized_train_Kappa = quadratic_weighted_kappa(y_train, val_optimized_train_pred_tuned)

        print(f"----> || Val Optimized Train QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {val_optimized_train_Kappa:.3f}{Style.RESET_ALL}")

        rounded_train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        rounded_val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)        

        train_S.append(train_Kappa)
        test_S.append(val_Kappa)
        
        #test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Rounded Train QWK: {rounded_train_kappa:.4f}, Rounded Validation QWK: {rounded_val_kappa:.4f}")
        #clear_output(wait=True)

        models_list.append(model)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    #tpm = test_preds.mean(axis=1)
    #tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    '''submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })'''

    return models_list #submission

In [ ]:
#Light = LGBMRegressor(**Params7, random_state=SEED, verbose=1, n_estimators=200)

In [ ]:
Params7 = {
    #'learning_rate': 0.03884249148676395,
    #'max_depth': 12,
    #'num_leaves': 413,
    #'min_data_in_leaf': 14,
    #'feature_fraction': 0.7987976913702801,
    #'bagging_fraction': 0.7602261703576205,
    #'bagging_freq': 2,
    #'lambda_l1': 4.735462555910575,
    #'lambda_l2': 4.735028557007343e-06
} # CV : 0.4094 | LB : 0.471

In [ ]:
Light = LGBMRegressor(n_estimators=200)

In [ ]:
RandomForestRegressor(random_state=SEED)

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 6,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 20,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Plain'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, train_final_df)#, test_final)

In [ ]:
def show_model_performances(models_list, data):
    X = data.drop(['id', 'sii'], axis=1)
    y = data['sii']
    
    for model in models_list:
        preds = model.predict(X)

        KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, preds),
                              method='Nelder-Mead')
        assert KappaOPtimizer.success, "Optimization did not converge."
        
        preds_tuned = threshold_Rounder(preds, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, preds_tuned)
    
        print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")            

In [ ]:
show_model_performances(catboost_models_list, train_final_df)

In [ ]:
def feature_importance(models_list, X):

    feature_names = X.columns
    feature_importance_list = []
    for model in models_list:
        feature_importance = model.get_feature_importance(type='FeatureImportance')
        feature_importance_list.append(feature_importance)

    importance_mean = np.mean(feature_importance_list, axis=0)
        
    # Create a DataFrame for better visualization
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance_mean
    }).sort_values(by='Importance', ascending=False)
    
    return importance_df

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, train_final_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
zero_importance_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
print(zero_importance_df.shape)
zero_importance_df

In [ ]:
zero_importance_cols = zero_importance_df['Feature'].tolist()
print(len(zero_importance_cols))
zero_importance_cols[:5]

In [ ]:
selected_df = train_final_df.drop(zero_importance_cols, axis=1)
print(selected_df.shape)
selected_df.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 7,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 20,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Plain'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, selected_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, selected_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
zero_importance_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
zero_importance_cols = zero_importance_df['Feature'].tolist()
selected_df = selected_df.drop(zero_importance_cols, axis=1)
print(selected_df.shape)
selected_df.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.01,
    'depth': 7,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 20,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Plain'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, selected_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, selected_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
zero_importance_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
zero_importance_cols = zero_importance_df['Feature'].tolist()
selected_df = selected_df.drop(zero_importance_cols, axis=1)
print(selected_df.shape)
selected_df.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.007,
    'depth': 7,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 20,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Plain'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, selected_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, selected_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
1/20

In [ ]:
catboost_importance_df['Importance'].describe(percentiles=[0.01, 0.03] + [(i+1)/20 for i in range(20)] + [0.97, 0.99])

In [ ]:
catboost_importance_df['Importance'].quantile(0.5)

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.5)]
print(unimportant_df.shape)
unimportant_df

In [ ]:
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df = selected_df.drop(unimportance_cols, axis=1)
print(selected_df.shape)
selected_df.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.01,
    'depth': 8,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 20,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Plain'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, selected_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, selected_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
catboost_importance_df['Importance'].describe(percentiles=[0.01, 0.03] + [(i+1)/20 for i in range(19)] + [0.97, 0.99])

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.5)]
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df = selected_df.drop(unimportance_cols, axis=1)
print(selected_df.shape)
selected_df.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.01,
    'depth': 8,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'colsample_bylevel': 0.8
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list = TrainML(CatBoost_Model, selected_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list, selected_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.5)]
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df2 = selected_df.drop(unimportance_cols, axis=1)
print(selected_df2.shape)
selected_df2.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list2 = TrainML(CatBoost_Model, selected_df2)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list2, selected_df2.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
catboost_importance_df['Importance'].describe(percentiles=[0.01, 0.03] + [(i+1)/20 for i in range(19)] + [0.97, 0.99])

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.5)]

In [ ]:
sns.histplot(unimportant_df['Importance'], kde=True)

In [ ]:
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df3 = selected_df2.drop(unimportance_cols, axis=1)
print(selected_df3.shape)
selected_df3.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list3 = TrainML(CatBoost_Model, selected_df3)#, test_final)

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list3 = TrainML(CatBoost_Model, selected_df3)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list3, selected_df3.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.5)]

In [ ]:
sns.histplot(unimportant_df['Importance'], kde=True)

In [ ]:
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df4 = selected_df3.drop(unimportance_cols, axis=1)
print(selected_df4.shape)
selected_df4.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list4 = TrainML(CatBoost_Model, selected_df4)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list4, selected_df4.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.1)]

In [ ]:
sns.histplot(unimportant_df['Importance'], kde=True)

In [ ]:
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df5 = selected_df4.drop(unimportance_cols, axis=1)
print(selected_df5.shape)
selected_df5.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list5 = TrainML(CatBoost_Model, selected_df5)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list5, selected_df5.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
sns.histplot(catboost_importance_df['Importance'], kde=True)

In [ ]:
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.1)]

In [ ]:
sns.histplot(unimportant_df['Importance'], kde=True)

In [ ]:
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df6 = selected_df5.drop(unimportance_cols, axis=1)
print(selected_df6.shape)
selected_df6.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list6 = TrainML(CatBoost_Model, selected_df6)#, test_final)

In [ ]:
feature_cols_df = pd.DataFrame({'selected_features': selected_df6.columns.tolist()})
print(feature_cols_df.shape)
feature_cols_df

In [ ]:
feature_cols_df.to_csv(path + 'selected_features.csv', index=False)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list6, selected_df6.drop(['id', 'sii'], axis=1))
unimportant_df = catboost_importance_df[catboost_importance_df['Importance'] <= catboost_importance_df['Importance'].quantile(0.1)]
unimportance_cols = unimportant_df['Feature'].tolist()
selected_df7 = selected_df6.drop(unimportance_cols, axis=1)
print(selected_df7.shape)
selected_df7.head()

In [ ]:
catboost_params = {
    'learning_rate': 0.02,
    'depth': 9,
    'iterations': 2000,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 50,  # Increase this value
    'task_type': 'GPU',
    'use_best_model': True,
    'early_stopping_rounds': 30,
    #'subsample': 0.8,
    'boosting_type': 'Ordered'
}

CatBoost_Model = CatBoostRegressor(
    **catboost_params
)

catboost_models_list7 = TrainML(CatBoost_Model, selected_df7)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list[0], train_final_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df.head()

In [ ]:
catboost_importance_df

In [ ]:
unimportant_features_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]

In [ ]:
unimportant_features = unimportant_features_df['Feature'].tolist()
print(len(unimportant_features))
unimportant_features[:5]

In [ ]:
selected_train_df = train_final_df.drop(unimportant_features, axis=1)
print(selected_train_df.shape)
selected_train_df.head()

In [ ]:
catboost_models_list = TrainML(CatBoost_Model, selected_train_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list[0], selected_train_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
unimportant_features_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
unimportant_features = unimportant_features_df['Feature'].tolist()
print(len(unimportant_features))
unimportant_features[:5]

In [ ]:
selected_train_df = selected_train_df.drop(unimportant_features, axis=1)
print(selected_train_df.shape)
selected_train_df.head()

In [ ]:
catboost_models_list = TrainML(CatBoost_Model, selected_train_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list[0], selected_train_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
unimportant_features_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
unimportant_features = unimportant_features_df['Feature'].tolist()
print(len(unimportant_features))
unimportant_features[:5]

In [ ]:
selected_train_df = selected_train_df.drop(unimportant_features, axis=1)
print(selected_train_df.shape)
selected_train_df.head()

In [ ]:
CatBoost_Params = {
    'learning_rate': 0.003,
    'depth': 8,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 1,
    'l2_leaf_reg': 10,  # Increase this value
    'task_type': 'GPU'
}

CatBoost_Model = CatBoostRegressor(
    **CatBoost_Params,
    use_best_model=True
    #custom_metric=[quadratic_weighted_kappa]#,  # Use custom QWK as an additional metric
    #eval_metric=qwk_wrapper       # Optionally use it as the primary evaluation metric
)

catboost_models_list = TrainML(CatBoost_Model, selected_train_df)#, test_final)

In [ ]:
catboost_importance_df = feature_importance(catboost_models_list[0], selected_train_df.drop(['id', 'sii'], axis=1))
print(catboost_importance_df.shape)
catboost_importance_df

In [ ]:
unimportant_features_df = catboost_importance_df[catboost_importance_df['Importance'] == 0]
unimportant_features = unimportant_features_df['Feature'].tolist()
print(len(unimportant_features))
unimportant_features[:5]

In [ ]:
selected_train_df = selected_train_df.drop(unimportant_features, axis=1)
print(selected_train_df.shape)
selected_train_df.head()

In [ ]:
CatBoost_Params = {
    'learning_rate': 0.001,
    'depth': 10,
    'iterations': 1000,
    'random_seed': SEED,
    'verbose': 1,
    'l2_leaf_reg': 10,  # Increase this value
    'task_type': 'GPU'
}

CatBoost_Model = CatBoostRegressor(
    **CatBoost_Params,
    use_best_model=True
    #custom_metric=[quadratic_weighted_kappa]#,  # Use custom QWK as an additional metric
    #eval_metric=qwk_wrapper       # Optionally use it as the primary evaluation metric
)

catboost_models_list = TrainML(CatBoost_Model, selected_train_df)#, test_final)

In [ ]:
train_one_hot = train_nonan.copy()
test_one_hot = test_merge.copy()

In [ ]:
for col in cat_cols:
    if 'PCIAT' not in col:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)        
        test_sr = test_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        test_one_hot[col] = test_sr
        concat_sr = pd.concat([train_sr, test_sr])
        unique_list = sorted(concat_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)
            test_one_hot[new_col_name] = pd.Series(test_one_hot[col] == v).astype(int)
    else:
        train_sr = train_one_hot[col].fillna('Missing').apply(str)
        train_one_hot[col] = train_sr
        unique_list = sorted(train_sr.unique().tolist())

        for v in unique_list:
            new_col_name = col + '_' + v
            train_one_hot[new_col_name] = pd.Series(train_one_hot[col] == v).astype(int)

train_one_hot.drop(cat_cols, axis=1, inplace=True)
test_one_hot.drop([col for col in cat_cols if 'PCIAT' not in col], axis=1, inplace=True)

In [ ]:
print(train_ts.shape)
train_ts.head()

In [ ]:
print(train_one_hot.shape)
train_one_hot.head()

In [ ]:
print(test_one_hot.shape)
test_one_hot.head()

In [ ]:
train_one_hot.drop('id', axis=1, inplace=True)
test_one_hot.drop('id', axis=1, inplace=True)

In [ ]:
train = train_one_hot[test_one_hot.columns.tolist() + ['sii']].copy()
test = test_one_hot.copy()

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

In [ ]:
concat_df = pd.concat([train[test.columns], test], ignore_index=True)
print(concat_df.shape)
concat_df.head()

In [ ]:
imputer = KNNImputer(n_neighbors=5)
imputed_data_concat = imputer.fit_transform(concat_df)

In [ ]:
concat_imputed = pd.DataFrame(imputed_data_concat, columns=concat_df.columns)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
concat_imputed.isna().sum().sum()

In [ ]:
concat_imputed = feature_engineering(concat_imputed)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
if np.any(np.isinf(concat_imputed)):
    print('inf in data')
    concat_imputed = concat_imputed.replace([np.inf, -np.inf], np.nan)

In [ ]:
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(concat_imputed)

In [ ]:
concat_imputed = pd.DataFrame(imputed_data, columns=concat_imputed.columns)
print(concat_imputed.shape)
concat_imputed.head()

In [ ]:
concat_imputed.isna().sum().sum()

In [ ]:
train_imputed = concat_imputed.head(train.shape[0]).reset_index(drop=True)
test_imputed = concat_imputed.tail(test.shape[0]).reset_index(drop=True)

In [ ]:
train_imputed['sii'] = train['sii']

In [ ]:
train_final = train_imputed.copy()
test_final = test_imputed.copy()

In [ ]:
print(train_final.shape)
print(test_final.shape)

# Model Training and Evaluation

- **Model Types**: Various models are used, including:
  - **LightGBM**: A gradient-boosting framework known for its speed and efficiency with large datasets.
  - **XGBoost**: Another powerful gradient-boosting model used for structured data.
  - **CatBoost**: Optimized for categorical features without the need for extensive preprocessing.
  - **Voting Regressor**: An ensemble model that combines the predictions of LightGBM, XGBoost, and CatBoost for better accuracy.
- **Cross-Validation**: Stratified K-Folds cross-validation is employed to split the data into training and validation sets, ensuring balanced class distribution in each fold.
- **Quadratic Weighted Kappa (QWK)**: The performance of the models is evaluated using QWK, which measures the agreement between predicted and actual values, taking into account the ordinal nature of the target variable.
- **Threshold Optimization**: The `minimize` function from `scipy.optimize` is used to fine-tune decision thresholds that map continuous predictions to discrete categories (None, Mild, Moderate, Severe).


In [ ]:
def TrainML(model_class, train_data, test_data):
    X = train_data.drop(['sii'], axis=1)
    y = train_data['sii']

    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(test_data), n_splits))

    for fold, (train_idx, test_idx) in enumerate(tqdm(SKF.split(X, y), desc="Training Folds", total=n_splits)):
        X_train, X_val = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[test_idx]

        model = clone(model_class)
        model.fit(X_train, y_train)

        y_train_pred = model.predict(X_train)
        y_val_pred = model.predict(X_val)

        oof_non_rounded[test_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[test_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(test_data)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': sample['id'],
        'sii': tpTuned
    })

    return submission


# Hyperparameter Tuning

- **LightGBM Parameters**: Hyperparameters such as `learning_rate`, `max_depth`, `num_leaves`, and `feature_fraction` are tuned to improve the performance of the LightGBM model. These parameters control the complexity of the model and its ability to generalize to new data.
- **XGBoost and CatBoost Parameters**: Similar tuning is applied for XGBoost and CatBoost, adjusting parameters such as `n_estimators`, `max_depth`, `learning_rate`, `subsample`, and `regularization` terms (`reg_alpha`, `reg_lambda`). These help in controlling overfitting and ensuring the model's robustness.

In [ ]:
Params7 = {
    'learning_rate': 0.03884249148676395,
    'max_depth': 12,
    'num_leaves': 413,
    'min_data_in_leaf': 14,
    'feature_fraction': 0.7987976913702801,
    'bagging_fraction': 0.7602261703576205,
    'bagging_freq': 2,
    'lambda_l1': 4.735462555910575,
    'lambda_l2': 4.735028557007343e-06
} # CV : 0.4094 | LB : 0.471

In [ ]:
Light = LGBMRegressor(**Params7, random_state=SEED, verbose=-1, n_estimators=200)

In [ ]:
Submission1 = TrainML(Light, train_final, test_final)
Submission1.to_csv('submission.csv', index=False)

Submission1